In [1]:
import os
import sys
from osgeo import ogr
from qgis.core import *
from qgis.gui import *
from qgis import processing

from qgis.PyQt.QtGui import QColor, QImage
from qgis.PyQt.QtCore import QSize, QBuffer, QIODevice

qgs = QgsApplication([], False)
qgs.initQgis()
print(QgsApplication.showSettings())

Application state:
QGIS_PREFIX_PATH env var:		C:/OSGEO4~1/apps/qgis
Prefix:		C:/OSGEO4~1/apps/qgis
Plugin Path:		C:/OSGEO4~1/apps/qgis/plugins
Package Data Path:	C:/OSGEO4~1/apps/qgis/.
Active Theme Name:	
Active Theme Path:	C:/OSGEO4~1/apps/qgis/./resources/themes\\icons/
Default Theme Path:	:/images/themes/default/
SVG Search Paths:	C:/OSGEO4~1/apps/qgis/./svg/
		C:/Users/Asus/AppData/Roaming/python3\profiles\default/svg/
User DB Path:	C:/OSGEO4~1/apps/qgis/./resources/qgis.db
Auth DB Path:	C:/Users/Asus/AppData/Roaming/python3\profiles\default/qgis-auth.db



In [2]:
covid_gpkg = "covid-pt-2020-04-26.gpkg"
conn = ogr.Open(covid_gpkg)
for i in conn:
    vlayer = QgsVectorLayer("{}|layername={}".format(covid_gpkg, i.GetName()), i.GetName(), "ogr")
    if not vlayer.isValid():
        print("Layer {} failed to load".format(i.GetName()))
    else:
        QgsProject.instance().addMapLayer(vlayer)
        print("Layer {} loaded".format(i.GetName()))

Layer concelho loaded
Layer distrito loaded
Layer raa_central_concelho loaded
Layer raa_central_ilha loaded
Layer raa_ocidental_concelho loaded
Layer raa_ocidental_ilha loaded
Layer raa_oriental_concelho loaded
Layer raa_oriental_ilha loaded
Layer ram_concelho loaded
Layer ram_ilha loaded
Layer layer_styles loaded
Layer confirmados_concelho loaded
Layer confirmados_distrito_ilha loaded
Layer situacao_epidemiologica loaded


In [3]:
distrito = QgsProject.instance().mapLayersByName('distrito')[0]
concelho = QgsProject.instance().mapLayersByName('confirmados_concelho')[0]
import matplotlib.pyplot as plt
import numpy as np
import io

def print_graph(date,ii):
    dic = {}
    for c in concelho.getFeatures():
        num = int(c["dico"])
        id = num/int(100)
        id = int(id)
        number = c[date]

        if id not in dic:
            if number == NULL:
                dic[id] = 0
            else :
                if str(type(number)) == '<class \'PyQt5.QtCore.QVariant\'>':
                    dic[id] = int(str(number))
                else:
                    dic[id] = number
        else:
            if number != NULL:
                if str(type(number)) == '<class \'PyQt5.QtCore.QVariant\'>':
                    dic[id] = dic[id] + int(str(number))
                else:
                    dic[id] = dic[id] + number

    ind = 1
    dic_index = {}
    for d in distrito.getFeatures():
        dic_index[ind] = d["distrito"]
        ind += 1

    dist = {}
    for d in distrito.getFeatures():
        cases = list(dic_index.keys())[list(dic_index.values()).index(d["distrito"])]
        case = dic[int(cases)]
        dist[d["distrito"]] = case 
        #print("Em {} há {} caso(s) confirmados".format(d["distrito"],case))
    
    x = dist.keys()
    y = dist.values()
    plt.figure(figsize=(27, 8))
    plt.scatter(x, y)
    plt.title("Número de casos no dia " + str(date))
    string = '{}.png'.format(ii)
    plt.savefig(string,format='png')
    plt.ylim(0, 8000)
    plt.plot()
    plt.close()
    

In [4]:
import pandas as pd
df = pd.read_csv("concelhos.csv")
cn = list(df)
col_name = cn[3:36]
col_name.append('26/04/2020')

['24/03/2020', '25/03/2020', '26/03/2020', '27/03/2020', '28/03/2020', '29/03/2020', '30/03/2020', '31/03/2020', '01/04/2020', '02/04/2020', '03/04/2020', '04/04/2020', '05/04/2020', '06/04/2020', '07/04/2020', '08/04/2020', '09/04/2020', '10/04/2020', '11/04/2020', '12/04/2020', '13/04/2020', '14/04/2020', '15/04/2020', '16/04/2020', '17/04/2020', '18/04/2020', '19/04/2020', '20/04/2020', '21/04/2020', '22/04/2020', '23/04/2020', '24/04/2020', '25/04/2020', '26/04/2020']


In [5]:
import imageio as io
frame = ['{}.png'.format(i) for i in range(len(col_name))]
ii = 0
for i in col_name:
    print_graph(i,ii)
    ii +=1
    
render = []
for ff in frame:
    render.append(io.imread(ff))

io.mimsave('evolution.gif', render, duration = 0.5)

import os
for i in frame:
    os.remove(i)